# Kisawazishi Rahisi cha Picha

Notebook hii inaonyesha jinsi ya kuainisha picha kwa kutumia mtandao wa neva uliokwisha kufundishwa.

**Unachojifunza:**
- Jinsi ya kupakia na kutumia modeli iliyokwisha kufundishwa
- Usindikaji wa awali wa picha
- Kufanya utabiri kwenye picha
- Kuelewa alama za kujiamini

**Matumizi:** Kutambua vitu kwenye picha (kama "paka", "mbwa", "gari", n.k.)

---


## Hatua ya 1: Ingiza Maktaba Zinazohitajika

Hebu tuingize zana tunazohitaji. Usijali ikiwa bado hujaelewa zote!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Hatua ya 2: Pakia Modeli Iliyojifunza Awali

Tutatumia **MobileNetV2**, mtandao wa neva ambao tayari umefunzwa kwa mamilioni ya picha.

Hii inaitwa **Transfer Learning** - kutumia modeli ambayo mtu mwingine ameifunza!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Hatua ya 3: Kazi za Msaidizi

Tujenge kazi za kupakia na kuandaa picha kwa ajili ya modeli yetu.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Hatua ya 4: Jaribu kwenye Picha za Mfano

Hebu tujaribu kuainisha baadhi ya picha kutoka mtandao!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Ainisha Kila Picha


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Hatua ya 5: Jaribu Picha Zako Mwenyewe!

Badilisha URL hapa chini na URL ya picha yoyote unayotaka kuainisha.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Nini Kimefanyika?

1. **Tumepakia modeli iliyofunzwa awali** - MobileNetV2 ilifundishwa kwa mamilioni ya picha
2. **Tumefanyia picha maandalizi** - Tumepunguza ukubwa na kuziandaa kwa ajili ya modeli
3. **Modeli imetoa utabiri** - Imetoa uwezekano wa madarasa 1000 ya vitu
4. **Tumetafsiri matokeo** - Tumebadilisha namba kuwa majina yanayoeleweka na binadamu

### Kuelewa Alama za Uhakika

- **90-100%**: Ina uhakika mkubwa (karibu sahihi kabisa)
- **70-90%**: Ina uhakika (pengine sahihi)
- **50-70%**: Ina uhakika kiasi (huenda sahihi)
- **Chini ya 50%**: Haina uhakika sana (haijulikani)

### Kwa nini utabiri unaweza kuwa si sahihi?

- **Picha kwa pembe au mwanga usio wa kawaida** - Modeli ilifundishwa kwa picha za kawaida
- **Vitu vingi kwenye picha** - Modeli inatarajia kitu kimoja kikuu
- **Vitu nadra** - Modeli inajua tu madarasa 1000
- **Picha zenye ubora wa chini** - Picha zisizo wazi au zenye pikseli nyingi ni ngumu zaidi


## 🚀 Hatua Zifuatazo

1. **Jaribu picha tofauti:**
   - Tafuta picha kwenye [Unsplash](https://unsplash.com)
   - Bonyeza kulia → "Nakili anwani ya picha" kupata URL

2. **Jaribu mambo mapya:**
   - Nini kinatokea na sanaa ya kufikirika?
   - Je, inaweza kutambua vitu kutoka pembe tofauti?
   - Inashughulikiaje vitu vingi kwa wakati mmoja?

3. **Jifunze zaidi:**
   - Chunguza [masomo ya Maono ya Kompyuta](../lessons/4-ComputerVision/README.md)
   - Jifunze jinsi ya kufundisha kifaa chako cha kutambua picha
   - Elewa jinsi CNNs (Mitandao ya Neva ya Mzunguko) inavyofanya kazi

---

## 🎉 Hongera!

Umejenga kifaa cha kutambua picha kwa kutumia mtandao wa neva wa kisasa!

Mbinu hii hiyo inatumika katika:
- Google Photos (kuandaa picha zako)
- Magari yanayojiendesha (kutambua vitu)
- Uchunguzi wa matibabu (kuchambua X-rays)
- Udhibiti wa ubora (kugundua kasoro)

Endelea kuchunguza na kujifunza! 🚀



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuchukuliwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
